<a href="https://colab.research.google.com/github/dilasko/barbot/blob/main/barbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pandas gradio google-genai

In [8]:
import pandas as pd
#Veri ve CSV okuma için Pandas kütüphanesi içe aktarılır.
import gradio as gr
# Web tabanlı kullanıcı arayüzü (UI) oluşturmak için Gradio kütüphanesi içe aktarılır.
#Daha basit kullanım için iki harflikısaltmasına tanımlandırılır.
import random
# Rastgele seçimler yapmak için kullanılır.Aynı malzemeler içeren kokteyllerden birini seçer.
import os
import re
import time
# Gemini API kütüphaneleri
from google import genai
# Gemini API'sine erişim için ana kütüphane.
from google.genai.errors import APIError

# ----------------------------------------------------------------------
# 1. VERİ YÜKLEME VE API İSTEMCİSİ AYARLARI
# ----------------------------------------------------------------------

# Veri setini yüklenir.
try:
    df = pd.read_csv('final_cocktails.csv')
    # 'final_cocktails.csv' dosyasını Pandas DataFrame'e yükler.
    df['ingredients_lower'] = df['ingredients'].astype(str).str.lower()
except FileNotFoundError:
    print("HATA: 'final_cocktails.csv' dosyası bulunamadı. Lütfen dosyayı Colab'e yüklediğinizden emin olun.")
    df = pd.DataFrame()

# Gemini API istemcisini başlatma
GEMINI_ENABLED = False
client = None
try:
    # 📌 BURAYI DÜZENLEYİN: API Anahtarınızı tırnak işaretleri içine yazın

    API_KEY_SABİT = 'Kendi keyinizi buraya yazın'
    # Hızlı ve kesin çözüm için anahtar doğrudan değişkene atanır.

    # Ortam değişkenini kontrol et
    api_key_env = os.environ.get('GEMINI_API_KEY')

    # Anahtar belirleme mantığı: Önce sabit anahtarı, sonra ortam değişkenini kullan
    if API_KEY_SABİT and API_KEY_SABİT != 'SİZİN_API_ANAHTARINIZI_BURAYA_YAZIN':
        api_key = API_KEY_SABİT
    elif api_key_env:
        api_key = api_key_env
    else:
        api_key = None # Hiçbir anahtar bulunamadı

    if api_key:
        client = genai.Client(api_key=api_key)
        GEMINI_ENABLED = True
        print("✅ Gemini API istemcisi başarıyla başlatıldı. Çeviri ve Görsel oluşturma aktif.")
    else:
        print("❌ Gemini API Anahtarı bulunamadı. Çeviri ve Görsel oluşturma devre dışı.")

except Exception as e:
    print(f"❌ Gemini API istemcisi başlatılamadı: {e}")

# ----------------------------------------------------------------------
# 2. GEMINI ÇEVİRİ VE GÖRSEL FONKSİYONLARI
# ----------------------------------------------------------------------

def safe_translate(prompt, model='gemini-2.5-flash'):
    """Genel, güvenli çeviri işlevi."""
    if not GEMINI_ENABLED:
        return None
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        return response.text.strip()
        # Cevap metnini döndürür.
    except Exception as e:
        return None

def translate_item(item_en, context_prompt):
    """Kısa, tek bir terimi çevirir."""
    result = safe_translate(
        prompt=f"Bu İngilizce terimi Türkçe'ye çevir: '{item_en}'. Çeviri yaparken sadece çevrilmiş terimi döndür. Bağlam: {context_prompt}"
    )
    return result if result else item_en

def translate_instructions(text_en, max_retries=3):
    """Kokteyl talimatlarını profesyonel bir dille çevirir. API hatasına karşı yeniden dener."""
    if not GEMINI_ENABLED:
        return f"**(Çeviri yapılamadı - Orijinal Metin)**:\n{text_en}"

    for attempt in range(max_retries):
        result = safe_translate(
            prompt=f" {text_en}"
        )

        # Sonucun varlığını ve hata mesajı içermediğini kontrol et
        if result and "Çeviri yapılamadı" not in result:
            return result

        # Eğer çeviri başarısız olursa, bir süre bekle ve tekrar dene
        if attempt < max_retries - 1:
            print(f"⚠️ Talimat çevirisi başarısız oldu (Deneme {attempt + 1}). 2 saniye bekleniyor...")
            time.sleep(2)

    # Tüm denemeler başarısız olursa orijinal metni döndür
    return f"**(Çeviri yapılamadı - Orijinal Metin)**:\n{text_en}"


def translate_ingredient_list(list_en):
    """Malzeme listesinin tamamını çevirir (daha iyi bağlam için)."""
    result = safe_translate(
        prompt=f"Aşağıdaki kokteyl malzeme listesini ve ölçülerini tam ve doğru Türkçe'ye çevir. Formatı (madde işaretleri) aynen koru. Sadece çevrilmiş listeyi döndür:\n{list_en}"
    )
    return result if result else f"**(Çeviri yapılamadı - Orijinal Liste)**:\n{list_en}"

def safe_translate_to_en(text_tr, model='gemini-2.5-flash'):
    """Türkçe metni İngilizce'ye çevirir. Başarısız olursa orijinali döndürür."""
    if not GEMINI_ENABLED:
        return text_tr

    prompt = f"Bu malzeme listesini/ifadeyi tamamen İngilizce'ye çevir. Sadece İngilizce çeviriyi döndür. İfade: '{text_tr}'"

    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        translated_text = response.text.strip().lower()
        if len(translated_text) > 0:
            print(f"🔄 Arama girdisi çevrildi: '{text_tr}' -> '{translated_text}'")
            return translated_text
        return text_tr
    except Exception as e:
        print(f"⚠️ Arama terimi çevrilemedi: {e}. Orijinal metinle devam ediliyor.")
        return text_tr

def generate_cocktail_image(cocktail_name, instructions_tr):
    """Verilen kokteyl için görsel oluşturur ve geçici bir dosyaya kaydeder."""
    if not GEMINI_ENABLED:
        return None

    prompt = (
        f"Profesyonel fotoğrafçılık kalitesinde, '{cocktail_name}' kokteylinin hazırlanışını veya "
        f"servis edildiği anı yansıtan, iştah açıcı ve şık bir fotoğrafını oluştur. "
        f"Tarif detayları: {instructions_tr[:200]}..."
    )

    try:
        print(f"🔄 Gemini ile görsel oluşturuluyor: {cocktail_name}...")
        result = client.models.generate_images(
            model='imagen-3.0-generate-002',
            prompt=prompt,
            config=dict(
                number_of_images=1,
                output_mime_type="image/jpeg",
                aspect_ratio="1:1"
            )
        )

        if result.generated_images:
            image_data = result.generated_images[0].image.image_bytes
            temp_filename = f"temp_cocktail_image_{random.randint(1000, 9999)}.jpg"
            with open(temp_filename, 'wb') as f:
                f.write(image_data)
            return temp_filename

    except APIError as e:
        print(f"⚠️ Görsel oluşturma API hatası: {e}")
        return None
    except Exception as e:
        print(f"⚠️ Görsel oluşturma hatası: {e}")
        return None

    return None

# ----------------------------------------------------------------------
# 3. ANA CHATBOT FONKSİYONU
# ----------------------------------------------------------------------

def kokteyl_tarifi_al(malzeme_girisi):
    """
    Kullanıcının girdiği malzemelerin TÜMÜNÜ (VE mantığı) içeren kokteyl tarifini döndürür.
    Regex ile kelime sınırlarını zorlayarak daha kesin eşleşme sağlar.
    """
    if df.empty:
        return "Veri seti yüklenemedi. Lütfen dosyanın adını kontrol edin.", None

    # 0. TÜRKÇE GİRDİYİ İNGİLİZCE'YE ÇEVİRME
    if GEMINI_ENABLED:
        processed_input = safe_translate_to_en(malzeme_girisi)
    else:
        processed_input = malzeme_girisi

    # 1. GİRDİYİ İŞLEME: Virgülle ayırma ve temizleme
    search_terms = [term.strip().lower() for term in processed_input.split(',') if term.strip()]

    if not search_terms:
        output_text = "⚠️ Lütfen bir malzeme ismi veya virgülle ayrılmış birden fazla malzeme giriniz."
        return output_text, None

    # 2. FİLTRE OLUŞTURMA: AND (VE) mantığı + REGEX ile kesin eşleşme
    filter_condition = pd.Series([True] * len(df))

    for term in search_terms:
        # REGEX: \b (kelime sınırı) ile aranan terimin tam bir kelime olarak geçmesini zorlar.
        regex_pattern = r'\b' + re.escape(term) + r'\b'

        filter_condition = filter_condition & df['ingredients_lower'].str.contains(
            regex_pattern,
            na=False,
            regex=True # Regex kullanımını etkinleştir
        )

    # 3. FİLTREYİ UYGULAMA
    filtered_df = df[filter_condition]

    if filtered_df.empty:
        terimler = ", ".join([f"'{t}'" for t in search_terms])
        output_text = f"❌ Girdiğiniz **TÜM** malzemeleri (**{terimler}**) tam olarak içeren bir kokteyl tarifi bulunamadı. Lütfen daha az veya farklı malzemeler deneyin. (Not: Arama hassasiyeti artırıldı.)"
        return output_text, None

    # Birden fazla sonuç varsa, rastgele birini seç
    selected_recipe = filtered_df.sample(n=1).iloc[0]

    name = selected_recipe['name']
    alcoholic_en = selected_recipe['alcoholic']
    category_en = selected_recipe['category']
    glass_en = selected_recipe['glassType']
    instructions_en = selected_recipe['instructions']
    ingredients_en = selected_recipe['ingredients'].strip("[]").replace("'", "")
    measures_en = selected_recipe['ingredientMeasures'].strip("[]").replace("'", "")
    thumbnail_url = selected_recipe['drinkThumbnail']

    # 4. TÜM ALANLARI ÇEVİRME
    if alcoholic_en == 'Alcoholic':
        alcoholic_tr = "Evet (Alkollü)"
    elif alcoholic_en == 'Non Alcoholic':
        alcoholic_tr = "Hayır (Alkolsüz)"
    else:
        alcoholic_tr = translate_item(alcoholic_en, "Alkollü olup olmadığını belirten bir durum.")

    category_tr = translate_item(category_en, "Kokteylin ait olduğu içecek kategorisi.")
    glass_tr = translate_item(glass_en, "Kokteylin servis edildiği bardak tipi.")

    # Malzeme Listesini Formatlama ve Çevirme
    try:
        ingredient_list_en = [
            f"• {m.strip()} {i.strip()}"
            for m, i in zip(measures_en.split(','), ingredients_en.split(','))
        ]
        ingredients_formatted_en = "\n".join(ingredient_list_en)
    except Exception:
        ingredients_formatted_en = f"Malzemeler: {ingredients_en}\nÖlçüler: {measures_en}"

    ingredients_formatted_tr = translate_ingredient_list(ingredients_formatted_en)
    # GÜNCELLENMİŞ ÇEVİRİ FONKSİYONUNU KULLAN
    instructions_tr = translate_instructions(instructions_en)

    # 5. GÖRSEL İŞLEME VE OLUŞTURMA
    image_path = None
    image_source_text = ""

    if pd.notna(thumbnail_url) and thumbnail_url:
         image_path = thumbnail_url
    elif GEMINI_ENABLED:
        image_path = generate_cocktail_image(name, instructions_tr)
        if image_path:
             image_source_text = "\n\n(Bu görsel, Gemini yapay zekası tarafından tarifinize özel olarak oluşturulmuştur.)"

    # SONUCU TÜRKÇE OLARAK BİÇİMLENDİRME
    output_text = f"""
    ✨ *Kokteyl Adı:* {name}
    ---
    🍸 *Kategori:* {category_tr}
    🧊 *Bardak Tipi:* {glass_tr}
    🥂 *Alkollü:* {alcoholic_tr}
    ---
    *🧪 Malzemeler ve Ölçüler:*
    {ingredients_formatted_tr}
    ---
    *📜 Yapılış (Türkçe Talimatlar):*
    {instructions_tr}
    {image_source_text}
    """

    return output_text.strip(), image_path

# ----------------------------------------------------------------------
# 4. GRADIO BLOCKS ARAYÜZÜ (Düzenli Çıktı)
# ----------------------------------------------------------------------

with gr.Blocks(title="🍸 Kokteyl Tarif Chatbot'u B-A-R*BOT*") as demo:
    gr.Markdown(
        """
        # 🍸 Kokteyl Tarif Chatbot'u B-A-R*BOT
        Elindeki bir veya birden fazla malzemeyi **Türkçe veya İngilizce** virgülle ayırarak gir, sistem sana **girdiğin tüm malzemeleri** içeren bir kokteyl tarifi sunsun!


        """
    )

    # Giriş ve Butonlar
    with gr.Row():
        ingredient_input = gr.Textbox(
            lines=1,
            placeholder="Virgülle ayırarak birden fazla malzeme girin (Örn: Cin, Limon veya Gin, Lime)",
            label="Hangi malzemelerle kokteyl tarifi istersiniz?"
        )
        submit_btn = gr.Button("Tarifi Bul", variant="primary", scale=0)

    # Çıktılar
    with gr.Row():
        recipe_output = gr.Textbox(
            label="Kokteyl Tarifi",
            scale=2,
            lines=25
        )

        image_output = gr.Image(
            label="Kokteyl Görseli",
            type="pil",
            scale=1
        )

    # Butonun eylemini tanımlama
    submit_btn.click(
        fn=kokteyl_tarifi_al,
        inputs=ingredient_input,
        outputs=[recipe_output, image_output]
    )

    # Örnekler (Çoklu giriş örnekleri eklendi)
    gr.Examples(
        examples=[
            ["Cin, Limon"],
            ["Rom, Limon"],
            ["Viski, Şeker, Angostura"],
            ["Votka"]
        ],
        inputs=ingredient_input
    )

# Colab'de çalıştırmak için launch() metodunu kullanın
demo.launch(share=True)

✅ Gemini API istemcisi başarıyla başlatıldı. Çeviri ve Görsel oluşturma aktif.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://330d7d7a9a267f37aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
